In [1]:
import tensorflow as tf
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

#from tensorflow_examples.models.pix2pix import pix2pix
from IPython.display import clear_output

In [2]:
IMAGE_PATH = "data/images"
MASK_PATH = "data/masks"
TEST_PATH = "test"
PREDICTIONS_PATH = "predictions"
AUTOTUNE = tf.data.experimental.AUTOTUNE
WIDTH, HEIGHT = 256, 256
BATCH_SIZE = 10

In [3]:
def load_data(image_path, mask_path):
    images = sorted(glob(f"{image_path}/*.png"))
    masks = sorted(glob(f"{mask_path}/*.png"))
    return images, masks

image_files, mask_files = load_data(IMAGE_PATH, MASK_PATH)

In [4]:
print(len(image_files), len(mask_files))
DATASET_SIZE = len(image_files)

2667 2667


In [5]:
dataset = tf.data.Dataset.from_tensor_slices((image_files, mask_files))

In [6]:
def normalize(input_image, input_mask):
    input_image = tf.cast(input_image, tf.float32)/ 255.0
    #input_mask = tf.cast(input_mask, tf.uint8)
    input_mask = tf.cast(input_mask, tf.float32)/ 255.0
    #input_image = input_image/255.0
    #input_mask -= 1
    
    return input_image, input_mask


def parse_image(image_file, mask_file):
    #print(image_file, mask_file)
    img = tf.io.read_file(image_file)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, size=(WIDTH, HEIGHT))
    msk = tf.io.read_file(mask_file)
    msk = tf.image.decode_png(msk, channels=3)
    msk = tf.image.resize(msk, size=(WIDTH, HEIGHT))
    
    img, msk = normalize(img, msk)
    
    return img, msk

In [7]:
dataset = dataset.map(parse_image).batch(BATCH_SIZE)
dataset = dataset.prefetch(buffer_size=AUTOTUNE)

In [8]:
def display(display_list):
    plt.figure(figsize=(WIDTH, HEIGHT))
    print(display_list)
    title = ['input image', 'true mask', 'predicted mask']
    
    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
        plt.axis('off')
    plt.show()
    
for image, mask in dataset.take(2):
    sample_image, sample_mask = image, mask
    
display([sample_image[1], sample_mask[1]])

[<tf.Tensor: shape=(256, 256, 3), dtype=float32, numpy=
array([[[8.61557901e-01, 5.19806981e-01, 1.46767765e-01],
        [8.80384505e-01, 5.44607818e-01, 1.85531557e-01],
        [8.98705602e-01, 5.67945778e-01, 2.13901654e-01],
        ...,
        [8.64407182e-01, 5.68030000e-01, 1.90946698e-01],
        [8.68964434e-01, 5.45932889e-01, 1.50589764e-01],
        [8.73973668e-01, 5.44316769e-01, 1.06410846e-01]],

       [[8.70672464e-01, 5.25459588e-01, 1.44446999e-01],
        [8.78339469e-01, 5.40548384e-01, 1.71775430e-01],
        [8.91352654e-01, 5.51891863e-01, 1.84160545e-01],
        ...,
        [8.73560071e-01, 5.74747264e-01, 2.30667889e-01],
        [8.74854445e-01, 5.62254906e-01, 1.88066795e-01],
        [8.63863349e-01, 5.40065885e-01, 1.17279410e-01]],

       [[8.74318302e-01, 5.26524186e-01, 1.36014089e-01],
        [8.83900106e-01, 5.40234387e-01, 1.61481306e-01],
        [8.95802677e-01, 5.51562488e-01, 1.71706498e-01],
        ...,
        [8.74969363e-01, 5.8445

In [9]:
print(dataset)

<PrefetchDataset shapes: ((None, 256, 256, 3), (None, 256, 256, 3)), types: (tf.float32, tf.float32)>


In [10]:
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Activation, Lambda, GlobalAveragePooling2D, concatenate
from tensorflow.keras.layers import UpSampling2D, Conv2D, Dropout, MaxPooling2D, Conv2DTranspose
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

In [11]:
def dice_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
    return K.mean( (2. * intersection + smooth) / (union + smooth), axis=0)

def dice_loss(in_gt, in_pred):
    return 1-dice_coef(in_gt, in_pred)

def iou_coef(y_true, y_pred, smooth=1):
  intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
  union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
  iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
  return iou

def iou_loss(in_gt, in_pred):
    return 1-iou_coef(in_gt, in_pred)

In [12]:
def get_model():
    in1 = tf.keras.layers.Input(shape=(HEIGHT, WIDTH, 3 ))

    conv1 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(in1)
    conv1 = Dropout(0.2)(conv1)
    conv1 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool1)
    conv2 = Dropout(0.2)(conv2)
    conv2 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool2)
    conv3 = Dropout(0.2)(conv3)
    conv3 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv3)
    pool3 = MaxPooling2D((2, 2))(conv3)

    conv4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool3)
    conv4 = Dropout(0.2)(conv4)
    conv4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv4)
    pool4 = MaxPooling2D((2, 2))(conv4)
    
    conv41 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool4)
    conv41 = Dropout(0.2)(conv41)
    conv41 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv41)
    
    up0 = concatenate([UpSampling2D((2, 2))(conv41), conv4], axis=-1)
    conv42 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(up0)
    conv42 = Dropout(0.2)(conv42)
    conv42 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv42)

    up1 = concatenate([UpSampling2D((2, 2))(conv42), conv3], axis=-1)
    conv5 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(up1)
    conv5 = Dropout(0.2)(conv5)
    conv5 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv5)
    
    up2 = concatenate([UpSampling2D((2, 2))(conv5), conv2], axis=-1)
    conv6 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(up2)
    conv6 = Dropout(0.2)(conv6)
    conv6 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv6)

    up2 = concatenate([UpSampling2D((2, 2))(conv6), conv1], axis=-1)
    conv7 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(up2)
    conv7 = Dropout(0.2)(conv7)
    conv7 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv7)
    segmentation = Conv2D(1, (1, 1), activation='sigmoid', name='seg')(conv7)

    model = Model(inputs=[in1], outputs=[segmentation])

    model.compile(optimizer='adam',loss = iou_loss, metrics=[iou_coef,'binary_accuracy'])
    
    return model

In [13]:
model = get_model()
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 256, 256, 32) 0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 32) 9248        dropout[0][0]                    
______________________________________________________________________________________________

In [14]:
def save_prediction(predicted, epoch):
    pred_img = tf.keras.preprocessing.image.array_to_img(predicted)
    pred_img.save(f"{PREDICTION_SAVE_PATH}/epoch-{epoch}.png")

def create_mask(pred_mask):
  pred_mask = tf.argmax(pred_mask, axis=-1)
  pred_mask = pred_mask[..., tf.newaxis]
  return pred_mask[0]

def show_predictions(dataset=None, num=2, is_save=False, epoch=0):
  if dataset:
    for image, mask in dataset.take(num):
      pred_mask = model.predict(image)
      created_mask = create_mask(pred_mask)
      display([image[0], mask[0], created_mask])
      if is_save:
        save_prediction(created_mask, epoch)
  else:
    display([sample_image, sample_mask,
             create_mask(model.predict(sample_image[tf.newaxis, ...]))])
    
#show_predictions(dataset, 2, True) 

In [15]:
train_size = int(0.9 * DATASET_SIZE)
val_size = int(0.1 * DATASET_SIZE)
test_size = int(0.1 * DATASET_SIZE)

print(train_size, val_size, test_size)

train_ds = dataset.take(train_size)
test_ds = dataset.skip(train_size)
#validation_ds = test_ds.skip(test_size)
test_ds = test_ds.take(test_size)

#print(train_ds, test_ds, validation_ds)
print(train_ds, test_ds)

2400 266 266
<TakeDataset shapes: ((None, 256, 256, 3), (None, 256, 256, 3)), types: (tf.float32, tf.float32)> <TakeDataset shapes: ((None, 256, 256, 3), (None, 256, 256, 3)), types: (tf.float32, tf.float32)>


In [16]:
class DisplayCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    clear_output(wait=True)
    epoch_num = epoch+1
    show_predictions(dataset, 2, True, epoch_num)
    print ('\nSample Prediction after epoch {}\n'.format(epoch_num))

In [17]:
print(train_ds)
print(test_ds)

<TakeDataset shapes: ((None, 256, 256, 3), (None, 256, 256, 3)), types: (tf.float32, tf.float32)>
<TakeDataset shapes: ((None, 256, 256, 3), (None, 256, 256, 3)), types: (tf.float32, tf.float32)>


In [18]:
print(model.input)
print(model.output)

KerasTensor(type_spec=TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'")
KerasTensor(type_spec=TensorSpec(shape=(None, 256, 256, 1), dtype=tf.float32, name=None), name='seg/Sigmoid:0', description="created by layer 'seg'")


In [ ]:
EPOCHS=3

steps_per_epoch = train_size//BATCH_SIZE
history = model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS, shuffle=True)

Epoch 1/3
267/267 [==============================] - 197s 737ms/step - loss: 0.0524 - iou_coef: 0.9475 - binary_accuracy: 0.3149
Epoch 2/3
169/267 [=================>............] - ETA: 1:11 - loss: 0.0650 - iou_coef: 0.9350 - binary_accuracy: 0.3099